<a href="https://colab.research.google.com/github/JishnuJayaraj/ML/blob/master/ML/NLP/KeywordExtraction/keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np 
import spacy
import gensim
# !matplotlib inline

import nltk
import string
import re
from collections import Counter

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

#stopwords=nltk.download('stopwords')                  #<-----
#punctuations = string.punctuation
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [2]:
# Read data from json file
df = pd.read_json('/content/drive/My Drive/RokinData/ToBeCleaned.json.gz')
df = df.head(20)



,all
0,A Phased Roll Out with Big Bang Elements: Oppo...
1,Democratizing Change Review with ThingWorx Nav...
2,PLM: Poorly Digitized Labs Give Dassault Systè...
3,Is Siemens’ New Low-Code Platform Mendix an “A...
4,How to Converge on One PLM System—The Tough Ap...


In [ ]:
# join heading and text column to single column
df.columns = ['date','heading','text','link','empty']
df['all']= df['heading'] + df['text']

df = df.drop(['date','heading','text','link','empty'], axis=1 )

df.head()

In [7]:
df['processed'] = df['all'].apply(lambda x: nlp(x))

def is_token_allowed(token):
     '''
         Only allow valid tokens which are not stop words
         and punctuation symbols.
     '''
     if (not token or not token.string.strip() or
         token.is_stop or token.is_punct):
         return False
     return True

def preprocess_token(token):
     # Reduce token to its lowercase lemma form
     return token.lemma_.strip().lower()


def cleaning(row):
  #print(d)
  
  doc = df['processed'].values[0]
  #doc = row['processed']         #  <-------------------------------- WHY

  #for word in doc:#
  filtered_tokens = [preprocess_token(token)
       for token in doc if is_token_allowed(token)]
  return filtered_tokens


# df['cleaned'] = df['all'].applymap(lambda x: cleaning(x) )
df['cleaned'] = df['processed'].apply(cleaning) #, also try to use map for efficiency

# Rejoining meaningful-stemmed to single snetence
def rejoin_words(row):
    my_list = row['cleaned']
    joined_words = ( " ".join(my_list))
    return joined_words

df['rejoined'] = df.apply(rejoin_words, axis=1)





# ------------------------------------------------------------------------------------
# capitalization
##df['all'] = df['all'].str.lower()
# Tokenize the column
##df['tokenized'] = df['all'].apply(lambda x: nlp(x))



df.head()
%whos

Variable           Type         Data/Info
-----------------------------------------
Counter            type         <class 'collections.Counter'>
cleaning           function     <function cleaning at 0x7f1c1b0b0950>
df                 DataFrame                             <...>\n\n[20 rows x 4 columns]
gensim             module       <module 'gensim' from '/u<...>ages/gensim/__init__.py'>
is_token_allowed   function     <function is_token_allowed at 0x7f1c13819ae8>
nlp                English      <spacy.lang.en.English object at 0x7f1c1b15af28>
nltk               module       <module 'nltk' from '/usr<...>ckages/nltk/__init__.py'>
np                 module       <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
pd                 module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
plt                module       <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
preprocess_token   function     <function preprocess_token at 0x7f1c226218c8>
re                 

In [8]:
freq_words = []
# Iterate through rows
for index, row in df.iterrows():
    text = row['cleaned']
    word_freq = Counter(text)
    freq_words.append(word_freq.most_common(5))

print(freq_words[1])

## ------------------------END ------------------------------

[('scania', 39), ('project', 25), ('dassault', 24), ('malmberg', 24), ('plm', 20)]


## Word Embeddings


In [ ]:
# Entity recognition
# df['new_col'] = df[2].apply(lambda x: list(nlp(x).ents))
#for entity in mydoc.ents:
  #print(entity.label_, ' | ', entity.text)

# df.head()
# POS tagging
#for token in doc:
    #print(token.text, token.pos_, token.tag_)

# NER tagging
#for token in doc:
    #print(token.text, token.ent_type_)

example_content = df.iloc[1]
print(example_content['tokenized'])

df.head()

print(string.punctuation)

kirigami designs hold thousands of times their own weightthe japanese art of origami (from ori, folding, and kami, paper) transforms flat sheets of paper into complex sculptures. variations include kirigami (from kiri, to cut), a version of origami that allows materials to be cut and reconnected using tape or glue.
but while both art forms are a source of ideas for science, architecture, and design, each has fundamental limitations. the flat folds required by origami result in an unlockable overall structure, while kirigami creations can't be unfolded back into their original, flattened states because of the adhesive.
taking inspiration from both art forms, researchers describe a new set of motifs for creating lightweight, strong, and foldable structures using soft materials. these kirigami structures can support 14,000 times their weight and, because they don't require adhesives or fasteners, can easily be flattened and refolded. published in physical review x, the work was conducted 

NameError: ignored

In [ ]:
complete_text = ('Gus Proto is a Python developer currently'
     'working for a London-based Fintech company. He is'
    ' interested in learning Natural Language Processing.'
     ' There is a developer conference happening on 21 July'
     ' 2019 in London. It is titled "Applications of Natural'
     ' Language Processing". There is a helpline number '
     ' available at +1-1234567891. Gus is helping organize it.'
    ' He keeps organizing local Python meetups and several'
     ' internal talks at his workplace. Gus is also presenting'
     ' a talk. The talk will introduce the reader about "Use'
    ' cases of Natural Language Processing in Fintech".'
     ' Apart from his work, he is very passionate about music.'
    ' Gus is learning to play the Piano. He has enrolled '
     ' himself in the weekend batch of Great Piano Academy.'
    ' Great Piano Academy is situated in Mayfair or the City'
    ' of London and has world-class piano instructors.')

complete_doc = nlp(complete_text)

# Remove stop words and punctuation symbols
words = [token.text for token in complete_doc
if not token.is_stop and not token.is_punct] 

word_freq = Counter(words)
# 5 commonly occurring words with their frequencies
common_words = word_freq.most_common(5)
print (common_words)

# Unique words
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)